**Modelo Sinapsia**

In [2]:
'''
Install:
- pandas
- numpy
- ssqueezepy
- mne
- scipy
- sklearn
- tensorflow
'''

'\nInstall:\n- pandas\n- numpy\n- ssqueezepy\n- mne\n- scipy\n- sklearn\n- tensorflow\n'

In [3]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Ordenando el dataset (no correr)**

----------> Juntamos los archivos

In [ ]:
#subo el archivo con las ids y diagnósticos (y)
metadata = pd.read_csv('metadata_guineabissau.csv')
metadata.head(5)

In [ ]:
metadata = metadata.drop('Remarks', axis=1)
metadata = metadata.drop('recordedPeriod', axis=1)
metadata = metadata.drop('startTime', axis=1)
metadata = metadata.drop('Eyes.condition', axis=1)

metadata.head(5)

In [ ]:
path = r'/content/eeg_data'
all_files = glob.glob(path + "/*.csv")

all_files = sorted(all_files)


def add_id_y (all_files, metadata):
  index = 0

  modified_dataframes_list = []
  non_eeg_columns = []

  for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    dataframe = df.iloc[:, 1:15].copy()
    dataframe['subject_id'] = metadata['subject.id'][index]
    dataframe['y'] = metadata['Group'][index]
    index += 1
    modified_dataframes_list.append(dataframe)

  return modified_dataframes_list

'''def join_csv (modified_dataframes_list):
  dataframes_list = []

  for filename in modified_dataframes_list:
    dataframe = pd.read_csv(filename, index_col=None, header=0)
    dataframes_list.append(dataframe)

  all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
  all_dataframes.to_csv('eeg_data.csv', index=False)

  return all_dataframes'''

In [ ]:
modified_dataframes = add_id_y(all_files, metadata)

dataframes_list = []

for dataframe in modified_dataframes:
  dataframes_list.append(dataframe)

all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
all_dataframes.to_csv('eeg_data.csv', index=False)

In [ ]:
all_dataframes

**---------------Procesando el dataset para el modelo (formato mne)---------------**

**Colmna y**

Epilepsia == 1

Normal == 0

In [7]:
eeg_data = pd.read_csv('C:\\Users\\47575909\\Desktop\\eeg_data.csv')
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,4297.948718,4186.153846,4132.820513,4106.666667,4076.410256,1,Epilepsy
1,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,4295.897436,4187.179487,4126.666667,4105.641026,4065.128205,1,Epilepsy
2,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,4301.025641,4188.205128,4123.076923,4103.076923,4063.589744,1,Epilepsy
3,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,4298.974359,4184.615385,4127.179487,4095.384615,4071.282051,1,Epilepsy
4,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,4295.384615,4182.564103,4128.205128,4090.769231,4064.102564,1,Epilepsy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850363,4211.794872,3862.051282,4139.487179,3670.256410,4327.179487,3662.564103,4557.435897,3894.358974,4238.974359,4189.743590,4010.256410,4126.153846,3830.769231,3883.589744,97,Control
3850364,4209.743590,3861.538462,4138.974359,3667.179487,4323.589744,3662.051282,4556.410256,3893.846154,4232.820513,4188.205128,4005.641026,4125.128205,3829.230769,3878.974359,97,Control
3850365,4211.282051,3862.564103,4136.410256,3671.282051,4324.102564,3664.102564,4556.923077,3896.923077,4235.384615,4185.128205,4005.128205,4123.076923,3829.230769,3882.051282,97,Control
3850366,4217.435897,3866.666667,4140.000000,3676.923077,4326.153846,3668.205128,4558.974359,3901.025641,4234.871795,4184.615385,4010.256410,4122.564103,3829.230769,3890.769231,97,Control


In [8]:
#paso los vaores categóricos a numéricos
mapping = {'Epilepsy': 1, 'Control': 0}
eeg_data['y'] = eeg_data['y'].map(mapping)
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,4297.948718,4186.153846,4132.820513,4106.666667,4076.410256,1,1
1,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,4295.897436,4187.179487,4126.666667,4105.641026,4065.128205,1,1
2,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,4301.025641,4188.205128,4123.076923,4103.076923,4063.589744,1,1
3,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,4298.974359,4184.615385,4127.179487,4095.384615,4071.282051,1,1
4,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,4295.384615,4182.564103,4128.205128,4090.769231,4064.102564,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850363,4211.794872,3862.051282,4139.487179,3670.256410,4327.179487,3662.564103,4557.435897,3894.358974,4238.974359,4189.743590,4010.256410,4126.153846,3830.769231,3883.589744,97,0
3850364,4209.743590,3861.538462,4138.974359,3667.179487,4323.589744,3662.051282,4556.410256,3893.846154,4232.820513,4188.205128,4005.641026,4125.128205,3829.230769,3878.974359,97,0
3850365,4211.282051,3862.564103,4136.410256,3671.282051,4324.102564,3664.102564,4556.923077,3896.923077,4235.384615,4185.128205,4005.128205,4123.076923,3829.230769,3882.051282,97,0
3850366,4217.435897,3866.666667,4140.000000,3676.923077,4326.153846,3668.205128,4558.974359,3901.025641,4234.871795,4184.615385,4010.256410,4122.564103,3829.230769,3890.769231,97,0


In [9]:
#función para convertir los datos a formato mne
def convertDF2MNE(sub):
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128)
    info.set_montage('standard_1020')
    data=mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    #data.filter(l_freq=0.1,h_freq=45)
    epochs = mne.make_fixed_length_epochs(data, duration = 3, overlap = 2)
    return epochs.get_data()

In [15]:
#hago una lista con cada id de cada subject
groups = eeg_data.groupby('subject_id')
grp_keys = list(groups.groups.keys())
grp_keys.pop(0)
print(grp_keys)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97]


In [17]:
from tqdm import tqdm

In [8]:
grpnos, labels, paths = [],[],[]
for i, grpno in tqdm(enumerate(grp_keys)):
    grp = groups.get_group(grpno).drop(['subject_id'],axis=1)
    label = int(grp['y'].unique())
    subject_id = grpno
    grp = grp.drop('y',axis=1)
    data = convertDF2MNE(grp)#(trials, channels, length)
    for c, x in enumerate(data):#loop trials
        Wx, scales = cwt(x, 'morlet')
        Wx=np.abs(Wx)
        path=os.path.join('./scaleogram',f'subvideo_{grpno}/',)
        os.makedirs(path,exist_ok=True)
        path=path+f'trial_{c}.npy'
        np.save(path, Wx)

        grpnos.append(i)
        labels.append(label)
        paths.append(path)

0it [00:00, ?it/s]

Creating RawArray with float64 data, n_channels=14, n_times=38528
    Range : 0 ... 38527 =      0.000 ...   300.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Not setting metadata
Not setting metadata
299 matching events found
No baseline correction applied
0 projection items activated
Loading data for 299 events and 384 original time points ...
0 bad epochs dropped


1it [00:19, 19.02s/it]

Creating RawArray with float64 data, n_channels=14, n_times=39808
    Range : 0 ... 39807 =      0.000 ...   310.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Not setting metadata
Not setting metadata
309 matching events found
No baseline correction applied
0 projection items activated
Loading data for 309 events and 384 original time points ...
0 bad epochs dropped


2it [00:36, 18.01s/it]

...

95it [28:59, 17.91s/it]

Creating RawArray with float64 data, n_channels=14, n_times=37632
    Range : 0 ... 37631 =      0.000 ...   293.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Not setting metadata
Not setting metadata
292 matching events found
No baseline correction applied
0 projection items activated
Loading data for 292 events and 384 original time points ...
0 bad epochs dropped


96it [29:16, 17.72s/it]

Creating RawArray with float64 data, n_channels=14, n_times=38400
    Range : 0 ... 38399 =      0.000 ...   299.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Not setting metadata
Not setting metadata
298 matching events found
No baseline correction applied
0 projection items activated
Loading data for 298 events and 384 original time points ...
0 bad epochs dropped


97it [29:34, 18.29s/it]


#### Modelo (CNN) 

In [17]:
#SEPARO EN X E Y

directory_path = r'C:\Users\47575909\Desktop\scaleogram' #Ruta de los archivos (carpeta scaleogram)

X = []
y = []

subfolders = [f.name for f in os.scandir(r'C:\Users\47575909\Desktop\scaleogram') if f.is_dir()]

for i, subject_id in enumerate(grp_keys):
    for c in range(len(subfolders)):
        file_path = f'{directory_path}/subvideo_{subject_id}/trial_{c + 97}.npy'
        data = np.load(file_path)
        X.append(data)  # agrego datos cwt a X
        y.append(labels[i])  # agrego las labels a y

#acá tengo la info de x e y separada
X = np.array(X)
y = np.array(y)

print('Ya se separó todo!')
print('Salió todo bien, tranquina :))')



#NOOOOOOOOOOOOOOOOOOOOOOO TENGO QUE USAR EL DATAREADER DEL INDIO DE YOUTUBE

NameError: name 'labels' is not defined

In [5]:
#NORMALIZO X en rango [0, 1]
X = (X - X.min()) / (X.max() - X.min())

AttributeError: 'list' object has no attribute 'min'

In [ ]:
#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

cwt.shape = (14, 230, 384)

- Alto: 230 (número de filas en tus imágenes CWT).
- Ancho: 384 (número de columnas en tus imágenes CWT).
- Canales: 14 (número de canales EEG).

In [ ]:
#si es que me toma mal la imput shape seguramente es por la longitud de la grabación de cada eeg, 
# por lo que dice la info del dataset no todos los eeg tienen la exacta misma duración

In [ ]:
#CREO LA CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(230, 384, 14)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
#COMPILO
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
#ENTRENO
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
test_loss, test_acc = model.evaluate(X_test, y_test)

#capaz que debería probar con cross validation después

In [ ]:
#VEO LA PERFORMANCE
print("Accuracy : ", test_acc)
print("Loss : ", test_loss)

**Tengo que hacer y notas**
##### **--> pasar todo a una pc**
##### **--> separar en datos de train, test y validation**

`78` = train

`10` = validation

`9` = test

##### **--> ver el formato y como feedearselo a una red neuronal**

Opciones de modelos a investigar:

*   `Convolutional neural network (CNN)`
*   `Recurrent neural network (RNN)`



**ChatGPT dice:**

Para la detección de epilepsia utilizando datos EEG, una de las arquitecturas de modelo que ha demostrado ser efectiva es una red neuronal convolucional (CNN) adaptada a la tarea de clasificación binaria. Dado que tienes representaciones de tus datos en formato de escala de tiempo-frecuencia (por ejemplo, mediante la transformada continua Wavelet - CWT), puedes aprovechar la capacidad de las CNN para capturar patrones espaciales y temporales en tus datos.

Aquí hay un enfoque general que puedes seguir:

1. **Preprocesamiento de datos**: Asegúrate de que tus datos estén preprocesados adecuadamente, incluyendo la extracción de las representaciones de escala de tiempo-frecuencia (escalogramas) que has mencionado.

2. **División de datos**: Divide tus datos en conjuntos de entrenamiento, validación y prueba, como se mencionó anteriormente.

3. **Diseño de la CNN**:
   - Diseña una arquitectura de CNN para la clasificación binaria (epilepsia o no epilepsia).
   - La entrada de la CNN será la representación de escala de tiempo-frecuencia de tus datos EEG.
   - La arquitectura puede constar de varias capas convolucionales seguidas de capas de agrupación (pooling) para capturar patrones espaciales y temporales en los escalogramas.
   - Agrega capas totalmente conectadas (densas) para realizar la clasificación final.

4. **Entrenamiento del modelo**:
   - Utiliza el conjunto de entrenamiento para entrenar la CNN.
   - Define una función de pérdida apropiada, como la entropía cruzada binaria (binary cross-entropy).
   - Ajusta los hiperparámetros, como la tasa de aprendizaje y la cantidad de épocas, y utiliza el conjunto de validación para la selección de modelos y el ajuste de hiperparámetros.

5. **Evaluación del modelo**:
   - Evalúa el rendimiento del modelo en el conjunto de prueba utilizando métricas como precisión, sensibilidad, especificidad y el área bajo la curva ROC (AUC-ROC).
   - Realiza un análisis adicional de las métricas de evaluación, como la matriz de confusión, para entender mejor el rendimiento del modelo en la detección de epilepsia.

6. **Ajuste y optimización**:
   - Si el rendimiento del modelo no es satisfactorio, considera realizar ajustes en la arquitectura de la CNN, la cantidad de capas, el tamaño del filtro, etc., y vuelve a entrenar y evaluar.

7. **Validación cruzada**:
   - Para obtener una evaluación más robusta, considera la posibilidad de realizar validación cruzada en lugar de una única división de los datos.

Recuerda que la calidad de tus datos, la cantidad de muestras y la representatividad de las clases (epilepsia y no epilepsia) son factores críticos para el éxito del modelo. Además, es importante consultar la literatura científica y posiblemente colaborar con expertos en neurociencia para asegurarte de que estás considerando adecuadamente los aspectos clínicos y científicos de la detección de epilepsia.